# Setup

In [1]:
import sys
from IPython.display import display, HTML
import torch

In [2]:
# Check if GPU is enabled (for Colab users)
in_colab = "google.colab" in sys.modules
if in_colab:
  if torch.cuda.is_available():
      device_name = torch.cuda.get_device_name(0)
      print(f"✅ GPU detected: {device_name}")
  else:
      warning_html = """
      <div style="background-color:#fff3cd;border:1px solid #ffeeba;
                  padding:15px;border-radius:8px;">
          ⚠️ <b>No GPU detected.</b><br>
          To speed up model inference:
          <ol>
              <li>Click on <b>Runtime</b> (top menu)</li>
              <li>Select <b>Change runtime type</b></li>
              <li>Set <b>Hardware accelerator</b> to <b>GPU</b></li>
              <li>Then re-run this notebook.</li>
          </ol>

          Or, you can continue to the next cells but it will take a while running on CPU... up to you!
      </div>
      """
      display(HTML(warning_html))
      device = torch.device("cpu")
      raise RuntimeError(
          "⚠️ No GPU detected. Please change: Runtime > Change runtime type > GPU."
      )

In [ ]:
if in_colab:
    # Clone GitHub repo and move into it
    !git clone https://github.com/championjeyson/agb-prediction.git
    %cd agb-prediction

    # Install dependencies
    !pip install -r requirements.txt

    # # Make sure src/ is importable
    # sys.path.append("src")

else:
    # Make sure src/ is importable
    sys.path.append("..")
    # sys.path.append("../src")

s:\Portfolio\AGB_using_AGBD


s:\Portfolio\AGB_using_AGBD\.venv\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


# Visualize some of the inputs

In [6]:
from src.inference import run_inference

ModuleNotFoundError: No module named 'src.inference'

# Run inference on the example dataset

In [4]:
from inference import run_inference
run_inference()

/content/agb-prediction/src/imagery_processing/__init__.py:114: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  xds_lc_encoded = xr.apply_ufunc(


Unfreeze last layer (mean regressor)... args.freeze_last_mean=False


Predicting patches: 100%|██████████| 100/100 [05:46<00:00,  3.46s/patch]


0